<a href="https://colab.research.google.com/github/NewCode321/ES327/blob/main/getFeatures_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


# References

## MediaPipe
- https://google.github.io/mediapipe/solutions/face_mesh.html#:~:text=MediaPipe%20Face%20Mesh%20is%20a,for%20a%20dedicated%20depth%20sensor.

## MHIs
- Paper: https://web.cse.ohio-state.edu/~davis.1719/Publications/event01.pdf
- Gihub Repositary: https://github.com/danielegiampaoli/MHI_Motion-History-Images


# Install Dependencies and Mount Google Drive

In [ ]:
# Install MediaPipe
!pip install mediapipe opencv-python

# Install Pims and AV
!pip install pims
!pip install av


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 31.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pims: filename=PIMS-0.6.1-py3-none-any.whl size=82634 sha256=53a91b50b7a1058db240421ace3162f755a3af094328d204fa1161af66e49ec2
  Stored in directory: /root/.cache/pip/wheels/c8/51/78/e8155864f53ff0c9e01fe656f64b55712ba60fb2a451757505
Successfully built pims
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 43.7 MB/s eta 0:00:00


In [ ]:
# Import Packages

#general
import pandas as pd
import numpy as np
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import os
import os.path as path # added to make organising directories easier

import glob #required for looping through files in directories

# MediaPipe Functions
mp_drawing = mp.solutions.drawing_utils #function to draw landmarks
mp_drawing_styles = mp.solutions.drawing_styles 
mp_hands = mp.solutions.hands #hand points
mp_face_mesh = mp.solutions.face_mesh #face points

# for MHIs
import scipy
from scipy import signal
import math
import matplotlib.pyplot as plt
from matplotlib import cm #colourmaps
from mpl_toolkits.mplot3d import Axes3D 
from skimage import draw as drw
from skimage import color, img_as_float, morphology
import skimage
import pims
from IPython.display import Image

In [ ]:
# mounting google drive to colab to access dataset

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Feature Extractor class 
takes time to run


In [ ]:
# Class to Extract MHI Feature Dataframe from Video Frame Images
class Features:  

    # Built in Attributes  
    # none     

    # The init method or constructor  - used to create new instances of classes
    def __init__(self,dataset,label_file_list):  

        # Instance Atrributes
        self.dataset = dataset
        self.label_file_list = label_file_list

    # The repr method - specifies how the class should interact with the print command
    def __repr__(self):
        return f"Features from {self.dataset}"

    #Methods ----------#

    # Display subdirectories in dataset directory
    def displayDirs(self):
      totalFiles = 0
      totalDir = 0
      label_dict = {}
      label_list = []

      label_num = [0,1,2,3,4] #five classes 

      for i in label_num:
        label_dict[i] = self.label_file_list[i]
        for base, dirs, files in os.walk(f"{self.dataset}{label_dict[i]}"):
          print(f'{base}')
          for directories in dirs:
              totalDir += 1
          for Files in files:
              totalFiles += 1

      print('Total number of files',totalFiles)
      print('Total Number of directories',totalDir)

    #Create a dictionary of video clips with: 0 - video clip directory path, 1 - frame file names list and 2 - label numbers
    def getClips(self):
      totalFiles = 0
      totalDir = 0
      label_dict = {}
      clip_dict = {}
      clips =  []
      frame_groups = []

      label_num = [0,1,2,3,4] # five classes
      
      for i in label_num:
        label_dict[i] = self.label_file_list[i]
        for base, dirs, files in os.walk(f"{self.dataset}{label_dict[i]}"):
          for directory in dirs:
              clips.append(f"clip{totalDir}")
              DIRECTORY_PATH = f'{self.dataset}{label_dict[i]}{directory}'
              frame_list = [f"{DIRECTORY_PATH}/", os.listdir(DIRECTORY_PATH),i]
              frame_groups.append(frame_list)
              totalDir += 1

      for clip in clips:
          for group in frame_groups:
              clip_dict[clip] = group
              frame_groups.remove(group)
              break 

      return clip_dict


    # Display first frames of each video clip without annotation     
    def displayFirstFrames(self):
      temp = self.getClips()

      firsts = []
      for clip in temp:
        # show the first frames in each video clip without annotation
        firsts.append(f"{temp[clip][0]}{temp[clip][1][0]}")
        for first in firsts:
          image = cv2.cvtColor(cv2.imread(first), cv2.COLOR_BGR2RGB)
          plt.rcParams["figure.figsize"] = (20,5)
          plt.subplot(1, len(firsts), firsts.index(first)+1)
          plt.title(f"Clip{firsts.index(first)+1}");plt.axis('off');
          plt.imshow(image)

    # Get Dataframe (hands, not MHIs)
    def getHandDF(self):
      temp = self.getClips()

      # Initialise the arrary
      rows = []

      for clip in temp:
        points = []
        IMAGE_FILES = glob.glob(f"{temp[clip][0]}*")
        with mp_hands.Hands(
          static_image_mode=True,
          max_num_hands=2,
          min_detection_confidence=0.5) as hands:
          for idx, file in enumerate(IMAGE_FILES):
            #Get the video clip label number e.g. 0 = agg
            label = temp[clip][2]

            # Read an image, flip it around y-axis for correct handedness output (see
            # above).
            image = cv2.flip(cv2.imread(file), 1)

            # Convert the BGR image to RGB before processing.
            results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            # Create a list of handedness value and xy coordinates for 21 key points
            landmarks_dict = results.multi_hand_landmarks
            # For images with no hands present set hand index to -1 (binary feature)
            if not landmarks_dict:
              hand_index = -1
            else: 
              results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
              landmarks_dict = results.multi_hand_landmarks
              key_point = landmarks_dict[0].landmark
              hand_index = results.multi_handedness[0].classification[0].index #left = 0, right = 1, both = 2, none = -1

            # Create a row of hand index and xy coordinates for each frame in the video
            point = []
            point.append(hand_index)

            for i in range(21):
              if not landmarks_dict:
                point.append([0,0])
              else: 
                point.append([key_point[i].x, key_point[i].y])
            point.append(label) #an "instance" = a list of 21 point coordinates) with index and label
            points.insert(idx,point) # for each frame in the video 

            # Reformating from samples*frames to samples*points

            # Create a temporary dataframe to get matrix transpose
            df = pd.DataFrame(points)
            x = df.iloc[:,1:22].T
            # get a row for each video clip as a list 
            row = x.values.tolist()

            # add the hand index and label values
            row.append(hand_index)
            row.append(label)
        
        rows.insert(idx,row) # videos down , points accross
      # Get list of column headers
      num_list = list(range(0, 21)) 
      columns = []
      for num in num_list:
        columns.append(f"hand_point{num}")
      columns.append("Hand Index")
      columns.append("Label")

      # create dataframe
      hand_df = pd.DataFrame(rows,columns = columns)
      return hand_df

    # Get Dataframe (face, not MHIs)
    def getFaceDF(self):
      temp = self.getClips()
      # Initialise the arrary
      rows = []
      
      for clip in temp:
        points = []
        IMAGE_FILES = glob.glob(f"{temp[clip][0]}*")
        with mp_face_mesh.FaceMesh(
            static_image_mode=True,
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5) as face_mesh:
            for idx, file in enumerate(IMAGE_FILES):
              #Get the video clip label number e.g. 0 = agg
              label = temp[clip][2]
              
              # Read an image, flip it around y-axis for correct handedness output (see
              # above).
              image = cv2.flip(cv2.imread(file), 1)
              
              # Convert the BGR image to RGB before processing.
              results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
              
              # Create a list of xy coordinates for 468 key points
              landmarks_dict = results.multi_face_landmarks
              # For images with no faces present set face index to -1 (binary feature)
              if not landmarks_dict:
                face_index = 0 # no faces detected
              else: 
                results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                landmarks_dict = results.multi_face_landmarks
                key_point = landmarks_dict[0].landmark
                face_index = 1 # face detected

              # Create a row of hand index and xy coordinates for each frame in the video
              point = []

              for i in range(468):
                if not landmarks_dict:
                  point.append([0,0])
                else: 
                  point.append([key_point[i].x, key_point[i].y])
              point.append(label) #an "instance" = a list of 21 point coordinates) with index and label
              points.insert(idx,point) # for each frame in the video 

              # Reformating from 7*50 to 7*468

              # Create a temporary dataframe to get matrix transpose
              df = pd.DataFrame(points)
              x = df.iloc[:,0:468].T 
              # get a row for each video clip as a list 
              row = x.values.tolist()

              # add the face index and label values
              row.append(face_index)
              row.append(label)
    
        rows.insert(idx,row) # videos down , points accross

      # Get list of column headers
      num_list = list(range(468)) 
      columns = []
      for num in num_list:
        columns.append(f"face_point{num}")
      columns.append("Face Index")
      columns.append("Label")
      
      # create dataframe
      face_df = pd.DataFrame(rows,columns = columns)
      return face_df

      # Function to get a full dataframe of hand and face points with labels
    def getFullDF(self):
       face_df = self.getFaceDF()
       hand_df = self.getHandDF()
        #hand points + face points + hand index + face index and video labels
       full_df = pd.concat([hand_df.iloc[:,0:21], face_df.iloc[:,0:468],hand_df.iloc[:,21],face_df.iloc[:,468:470]], axis=1, join="inner")
       return full_df
    
    # Function to return motion history image histograms of all points
    def getMHIHists(self):
      full_df = self.getFullDF()
      # get the point frames and labels for each clip 
      clips = []
      for iter_c in range(len(full_df)):
        frames = []
        labels = []#new
        for iter_f in range(len(full_df.iloc[iter_c,0:489][0])):
          points = []
          for iter_p in range(len(full_df.iloc[iter_c,0:489])):
            point = full_df.iloc[iter_c,iter_p][iter_f]#change to make a frame of the correct size for this first frame, then make iterate for all frames print(points)
            points.append(point)

          #convert to numpy array
          np_points = [np.array(point) for point in points]

          frame = np.zeros((650, 800),dtype=int)#might need to change int to float#should be 480,640 but bad normalisation
          X, Y = [(i[0])*640 for i in np_points], [(i[1])*480 for i in np_points]
          for i in range(len(np_points)):
            frame[round(Y[i]), round(X[i])] = 1
          frames.append(frame)
        clips.append(frames)
        labels.append(full_df.iloc[:,-1])

      # generate list of MHIs
      point_mhis = list()
      frame_nums = list()
      for i in range(len(clips)):
        # import clip frames
        v = clips[i] #frames in clip3. 1 3 4 5 
        T = len(v) # number of extracted frames

        # compute the frame difference images D
        tau = T 
        D_list = list()
        D_list.append(np.zeros_like(v[0]))
        for t in range(1,T):
            D = (img_as_float(color.rgb2gray(v[t])) - img_as_float(color.rgb2gray(v[t-1])))
            D_list.append(D)
        
        # compute the Psi images
        csi = 0.09
        Psi_list = list()
        Psi_list.append(v[0])
        for t in range(1,T):    
            Psi_tmp = (np.abs(D_list[t]) > csi).astype(int) + v[t]
            Psi = (Psi_tmp != 0).astype(int)
            Psi_list.append(Psi)
        
        # compute the Motion Energy Images E

        E_list = list()
        for t in range(tau-1,T):
            tmp = np.zeros_like(Psi_list[0])
            for ix in range(t-tau+1,t+1):
                tmp += Psi_list[ix]
            E = (tmp != 0).astype(int)
            E_list.append(E)
        
        # compute the motion history images H

        delta = 1
        H_list = list()
        H_list.append(v[0])#get the first frame
        for t in range(T):
            H = (Psi_list[t]*tau) + (1-Psi_list[t])*np.maximum(0,H_list[t-1]-delta)
            H_list.append(H)
        
        # create a list of the point_mhis
        point_mhis.append(H_list[T-2])
        frame_nums.append(T)

        # get MHI histograms!!!! this should be seperate!!!!!!!
        i = 1
        vectors = list()
        for mhi in point_mhis:
          # set bin edges so we have 30 bins
          edges = np.linspace(0,frame_nums[i-1],31)

          # mhi histograms 
          vector, edges = np.histogram(point_mhis[i-1], bins = edges)
          vectors.append(vector)

          #increase counter
          i +=1

      # Get list of column headers
      num_list = list(range(30)) 
      columns = []
      for num in num_list:
        columns.append(f"Descriptor{num}")
      
      # create dataframe
      mhi_hist_df = pd.DataFrame(vectors,columns = columns)
      mhi_hist_df['Labels'] = np.array(labels).T

      return mhi_hist_df 

    # Funtion to display MHIs of each video clip - has histograms in it also fro some reason 
    def displayMHIs(self):
          full_df = self.getFullDF()
          # get the point frames and labels for each clip 
          clips = []
          for iter_c in range(len(full_df)):
            frames = []
            labels = []#new
            for iter_f in range(len(full_df.iloc[iter_c,0:489][0])):
              points = []
              for iter_p in range(len(full_df.iloc[iter_c,0:489])):
                point = full_df.iloc[iter_c,iter_p][iter_f]#change to make a frame of the correct size for this first frame, then make iterate for all frames print(points)
                points.append(point)

              #convert to numpy array
              np_points = [np.array(point) for point in points]

              frame = np.zeros((650, 800),dtype=int)#might need to change int to float#should be 480,640 but bad normalisation
              X, Y = [(i[0])*640 for i in np_points], [(i[1])*480 for i in np_points]
              for i in range(len(np_points)):
                frame[round(Y[i]), round(X[i])] = 1
              frames.append(frame)
            clips.append(frames)
            labels.append(full_df.iloc[:,-1])

          # generate list of MHIs
          point_mhis = list()
          frame_nums = list()
          for i in range(len(clips)):
            # import clip frames
            v = clips[i] #frames in clip3. 1 3 4 5 
            T = len(v) # number of extracted frames

            # compute the frame difference images D
            tau = T 
            D_list = list()
            D_list.append(np.zeros_like(v[0]))
            for t in range(1,T):
                D = (img_as_float(color.rgb2gray(v[t])) - img_as_float(color.rgb2gray(v[t-1])))
                D_list.append(D)
            
            # compute the Psi images
            csi = 0.09
            Psi_list = list()
            Psi_list.append(v[0])
            for t in range(1,T):    
                Psi_tmp = (np.abs(D_list[t]) > csi).astype(int) + v[t]
                Psi = (Psi_tmp != 0).astype(int)
                Psi_list.append(Psi)
            
            # compute the Motion Energy Images E

            E_list = list()
            for t in range(tau-1,T):
                tmp = np.zeros_like(Psi_list[0])
                for ix in range(t-tau+1,t+1):
                    tmp += Psi_list[ix]
                E = (tmp != 0).astype(int)
                E_list.append(E)
            
            # compute the motion history images H

            delta = 1
            H_list = list()
            H_list.append(v[0])#get the first frame
            for t in range(T):
                H = (Psi_list[t]*tau) + (1-Psi_list[t])*np.maximum(0,H_list[t-1]-delta)
                H_list.append(H)
            
            # create a list of the point_mhis
            point_mhis.append(H_list[T-2])
            frame_nums.append(T)
          
          # Display all the MHIs in a subplot
          i = 1
          vectors = list()
          for mhi in point_mhis:
            plt.rcParams["figure.figsize"] = (20,5)
            plt.subplot(1, len(point_mhis), i)
            plt.title(f"MHI {i}");plt.axis('off');
            plt.imshow(mhi,cmap='jet')# Jet colourmap used to show point density clearly

            #---------------------!
            # set bin edges so we have 30 bins
            edges = np.linspace(0,frame_nums[i-1],31)

            # mhi histograms 
            vector, edges = np.histogram(point_mhis[i-1], bins = edges)
            x = (edges[1:]+edges[:-1])/2 #bin centers for plotting
            vectors.append(vector)

            #increase counter
            i +=1

    # Function to get MHI images 
    def getPointMHIs(self):
      full_df = self.getFullDF()
      # get the point frames and labels for each clip 
      clips = []
      #---starts: for each video clip create a list of frames showing only key points of size 650*800 and save to the list "clips", also save video class labels to "labels list"
      for iter_c in range(len(full_df)):
        frames = []
        labels = []
        for iter_f in range(len(full_df.iloc[iter_c,0:489][0])):
          points = []
          for iter_p in range(len(full_df.iloc[iter_c,0:489])):
            point = full_df.iloc[iter_c,iter_p][iter_f]
            points.append(point)

          #convert to numpy array
          np_points = [np.array(point) for point in points]

          frame = np.zeros((650, 800),dtype=int) # should be 480,640 
          X, Y = [(i[0])*640 for i in np_points], [(i[1])*480 for i in np_points]
          for i in range(len(np_points)):
            frame[round(Y[i]), round(X[i])] = 1
          frames.append(frame)
        clips.append(frames)
        labels.append(full_df.iloc[:,-1])
      #---ends: for each video clip create a list of frames showing only key points of size 650*800 and save to the list "clips", also save video class labels to "labels list"

      #---starts: use frames to generate list of MHIs for each video clip
      point_mhis = list()
      frame_nums = list()
      for i in range(len(clips)):
        # import clip frames
        v = clips[i] #frames in clip3. 1 3 4 5 
        T = len(v) # number of extracted frames

        # compute the frame difference images D
        tau = T 
        D_list = list()
        D_list.append(np.zeros_like(v[0]))
        for t in range(1,T):
            D = (img_as_float(color.rgb2gray(v[t])) - img_as_float(color.rgb2gray(v[t-1])))
            D_list.append(D)
        
        # compute the Psi images
        csi = 0.09
        Psi_list = list()
        Psi_list.append(v[0])
        for t in range(1,T):    
            Psi_tmp = (np.abs(D_list[t]) > csi).astype(int) + v[t]
            Psi = (Psi_tmp != 0).astype(int)
            Psi_list.append(Psi)
        
        # compute the Motion Energy Images E
        E_list = list()
        for t in range(tau-1,T):
            tmp = np.zeros_like(Psi_list[0])
            for ix in range(t-tau+1,t+1):
                tmp += Psi_list[ix]
            E = (tmp != 0).astype(int)
            E_list.append(E)
        
        # compute the motion history images H
        delta = 1
        H_list = list()
        H_list.append(v[0])#get the first frame
        for t in range(T):
            H = (Psi_list[t]*tau) + (1-Psi_list[t])*np.maximum(0,H_list[t-1]-delta)
            H_list.append(H)
        
        # create a list of the point_mhis
        point_mhis.append(H_list[T-2])
        frame_nums.append(T)
      #---ends: use frames to generate list of MHIs for each video clip

      # Return a list of point mhis
      return point_mhis

# Feature Extractor Driver Code

In [ ]:
# Set Inputs
dataset = '/content/gdrive/MyDrive/small_data/'#match filepath of images (videoframes in folders for each video segment in folders for each class)

label_file_list = ["0_int/","1_thi/","2_uns/","3_sur/", "4_hap/"]# match this to the file directories


# Create an Instance of the Features class
test = Features(dataset,label_file_list)

'''
Uncomment to run one at a time
'''

# # Display the directories in the dataset file
# test.displayDirs()

# # Get dictionary of the frame files for each clip and their labels
# test.getClips()

# # Display the first frames in each video clip without annotation
# test.displayFirstFrames()  

# Get a Dataframe with structure: rows for each clip, 492 columns (face and hand points) containing a point's coordinates for each frame (no MHI)
test.getFullDF()

# # Get a Dataframe of the point mhi histograms with their labels
# test.getMHIHists()

# # Display the generated MHI frames
# test.displayMHIs()

# # get face and hand point MHI image matrices dataframe
# test.getPointMHIs()

#add getStDevs() function

,hand_point0,hand_point1,hand_point2,hand_point3,hand_point4,hand_point5,hand_point6,hand_point7,hand_point8,hand_point9,...,face_point461,face_point462,face_point463,face_point464,face_point465,face_point466,face_point467,Hand Index,Face Index,Label
0,"[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...",...,"[[0.4965996742248535, 0.31040158867836], [0.49...","[[0.4961130619049072, 0.3137160539627075], [0....","[[0.5140247941017151, 0.28141412138938904], [0...","[[0.510439395904541, 0.28144487738609314], [0....","[[0.5078430771827698, 0.28138282895088196], [0...","[[0.5374106168746948, 0.28655219078063965], [0...","[[0.541528582572937, 0.28447577357292175], [0....",-1,1,0
1,"[[0.49709710478782654, 0.8215284943580627], [0...","[[0.5117023587226868, 0.7940971255302429], [0....","[[0.5328081846237183, 0.7791500687599182], [0....","[[0.5527566075325012, 0.7714900970458984], [0....","[[0.5675399303436279, 0.766732394695282], [0.5...","[[0.535225510597229, 0.7761654257774353], [0.5...","[[0.5589685440063477, 0.7485855221748352], [0....","[[0.5719754099845886, 0.729156494140625], [0.5...","[[0.5814388990402222, 0.7147162556648254], [0....","[[0.5370717644691467, 0.7789297103881836], [0....",...,"[[0.3690361976623535, 0.25159257650375366], [0...","[[0.36979904770851135, 0.25640928745269775], [...","[[0.3970301151275635, 0.21088118851184845], [0...","[[0.392649382352829, 0.21197208762168884], [0....","[[0.38934525847435, 0.21259494125843048], [0.3...","[[0.43141043186187744, 0.215709388256073], [0....","[[0.4376152753829956, 0.2121371030807495], [0....",1,1,0
2,"[[0.45501503348350525, 0.7680935263633728], [0...","[[0.4463120400905609, 0.7291788458824158], [0....","[[0.42269429564476013, 0.7014383673667908], [0...","[[0.3977319002151489, 0.688961386680603], [0.4...","[[0.3763104975223541, 0.6861835718154907], [0....","[[0.4243198335170746, 0.6894711256027222], [0....","[[0.3944441080093384, 0.6519860625267029], [0....","[[0.36857351660728455, 0.6407966613769531], [0...","[[0.3468656837940216, 0.6376463174819946], [0....","[[0.42227640748023987, 0.7108439207077026], [0...",...,"[[0.6071641445159912, 0.28219980001449585], [0...","[[0.6048367619514465, 0.2845645248889923], [0....","[[0.6179423928260803, 0.23443356156349182], [0...","[[0.6159443855285645, 0.23558330535888672], [0...","[[0.6157891750335693, 0.23655124008655548], [0...","[[0.6451289653778076, 0.24248185753822327], [0...","[[0.6492809653282166, 0.2403232753276825], [0....",1,1,0
3,"[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...",...,"[[0.4057295322418213, 0.44367915391921997], [0...","[[0.40805497765541077, 0.44649460911750793], [...","[[0.41965603828430176, 0.3859096169471741], [0...","[[0.41615718603134155, 0.3901999890804291], [0...","[[0.4134471118450165, 0.3939134478569031], [0....","[[0.44910067319869995, 0.3702944815158844], [0...","[[0.4526391923427582, 0.36583226919174194], [0...",-1,1,0
4,"[[0.5586010813713074, 0.9398457407951355], [0....","[[0.5337899923324585, 0.8746469616889954], [0....","[[0.49900496006011963, 0.8413518071174622], [0...","[[0.

In [ ]:
mhis = test.getPointMHIs()

# Save Extracted Data
uncomment to save files

In [ ]:
# # Export mhi_hist_df to csv file 
# mhi_hist_df = test.getMHIHists()
# mhi_hist_df.to_csv('/content/gdrive/MyDrive/mhi_hists2.csv')

<ipython-input-5-bd83f9964da3>:283: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  D = (img_as_float(color.rgb2gray(v[t])) - img_as_float(color.rgb2gray(v[t-1])))


In [ ]:
# point_df = test.getFullDF()
# point_df.to_csv('/content/gdrive/MyDrive/small_point_df.csv')

# Get point mhi image files and save mhi images

In [ ]:
# Export mhi_df to csv file THIS WORKS! :-)
full_df = pd.read_csv('/content/gdrive/MyDrive/point_df.csv')

In [ ]:
full_df = full_df.iloc[:,1:-3]# or load csv file

# Convert the contents of each cell back to a list 
import ast

def toList(string_list):
  return ast.literal_eval(string_list)

full_df = full_df.applymap(toList)#new_df later


In [ ]:
full_df

In [ ]:
# Seperate script to export mhi_df to csv 
full_df = pd.read_csv('/content/gdrive/MyDrive/point_df.csv')
full_df = full_df.iloc[:,1:]

# get the point frames and labels for each clip 
clips = []
#---starts: for each video clip create a list of frames showing only key points of size 650*800 and save to the list "clips", also save video class labels to "labels list"
for iter_c in range(len(full_df)):
  frames = []
  labels = []#new
  for iter_f in range(len(full_df.iloc[iter_c,0:489][0])):
    points = []
    for iter_p in range(len(full_df.iloc[iter_c,0:489])):
      point = full_df.iloc[iter_c,iter_p][iter_f]#change to make a frame of the correct size for this first frame, then make iterate for all frames print(points)
      points.append(point)

    #convert to numpy array
    np_points = [np.array(point) for point in points]

    frame = np.zeros((650, 800),dtype=int) # should be 480,640 
    X, Y = [(i[0])*640 for i in np_points], [(i[1])*480 for i in np_points]
    for i in range(len(np_points)):
      frame[round(Y[i]), round(X[i])] = 1
    frames.append(frame)
  clips.append(frames)
  labels.append(full_df.iloc[:,-1])
#---ends: for each video clip create a list of frames showing only key points of size 650*800 and save to the list "clips", also save video class labels to "labels list"

#---starts: use frames to generate list of MHIs for each video clip
point_mhis = list()
frame_nums = list()
for i in range(len(clips)):
  # import clip frames
  v = clips[i] #frames in clip3. 1 3 4 5 
  T = len(v) # number of extracted frames

  # compute the frame difference images D
  tau = T 
  D_list = list()
  D_list.append(np.zeros_like(v[0]))
  for t in range(1,T):
      D = (img_as_float(color.rgb2gray(v[t])) - img_as_float(color.rgb2gray(v[t-1])))
      D_list.append(D)
  
  # compute the Psi images
  csi = 0.09
  Psi_list = list()
  Psi_list.append(v[0])
  for t in range(1,T):    
      Psi_tmp = (np.abs(D_list[t]) > csi).astype(int) + v[t]
      Psi = (Psi_tmp != 0).astype(int)
      Psi_list.append(Psi)
  
  # compute the Motion Energy Images E
  E_list = list()
  for t in range(tau-1,T):
      tmp = np.zeros_like(Psi_list[0])
      for ix in range(t-tau+1,t+1):
          tmp += Psi_list[ix]
      E = (tmp != 0).astype(int)
      E_list.append(E)
  
  # compute the motion history images H
  delta = 1
  H_list = list()
  H_list.append(v[0])#get the first frame
  for t in range(T):
      H = (Psi_list[t]*tau) + (1-Psi_list[t])*np.maximum(0,H_list[t-1]-delta)
      H_list.append(H)
  
  # create a list of the point_mhis
  point_mhis.append(H_list[T-2])
  frame_nums.append(T)
#---ends: use frames to generate list of MHIs for each video clip

In [ ]:
point_mhis

In [ ]:
from PIL import Image
image = Image.fromarray(mhis[40].astype('uint8'))
plt.imshow(image)

In [ ]:
# Save each matrix as a PNG image file
from PIL import Image

for i, mhi in enumerate(mhis):
    # Convert the matrix to an image
    image = Image.fromarray(mhi.astype('uint8'))
    # Save the image as a file
    image.save(f'/content/gdrive/MyDrive/mhi_dataset/mhi_{i}.png')